CS 178 Project <br>
Team 69 - "Adulting" <br>
Members:<br>
Dingyi Chang #89167697<br>
Nicholas King #33099364<br>
Jenny My Lee #<br>

We will first download the data (the Adult dataset from UCI ML Repo) as well as importing any necessary libraries.

In [167]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import pandas as pd

#Read training data from adult.data and make Xtr and Ytr
data = np.array(pd.read_csv('adult.data'))
Xtr = data[:, :14]
Ytr = data[:, 14]

#Read testing data from adult.test and make Xte and Yte
data = np.array(pd.read_csv('adult.test', skiprows = 1))
Xte = data[:, :14]
Yte = data[:, 14]



